In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import PIL.Image

In [2]:
class_names = ['forward', 'stop', 'forward left', 'forward right']
FILES_PATH='/home/kostas/Autonomous_car/files/'

In [3]:
loaded_model = models.resnet18(pretrained=True)
loaded_model.fc = torch.nn.Linear(512, len(class_names))
device = torch.device('cuda')
loaded_model = loaded_model.to(device)

In [4]:
loaded_model.load_state_dict(torch.load(FILES_PATH + 'first_model.pt'))
loaded_model = loaded_model.eval()

In [5]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

# def preprocess(image):
#     image = PIL.Image.open(image)
#
#     image = transforms.functional.resize(image, (256, 256))
#     image = transforms.functional.to_tensor(image).to(device)
#     image.sub_(mean[:, None, None]).div_(std[:, None, None])
#     return image[None, ...]

def preprocess(image):
    image = PIL.Image.open(image)

    preprocess1 = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )])

    img_t = preprocess1(image)
    return torch.unsqueeze(img_t, 0)


In [24]:
test_path = '/home/kostas/Autonomous_car/files/track1/direction1_75/2020-12-17_17-43-06/img107.jpg'

In [25]:
res = loaded_model(preprocess(test_path).to(device=device))
res

tensor([[-0.7482,  1.0248, -1.2014, -1.1951]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [26]:
_, index = torch.max(res, 1)



In [27]:
percentage = torch.nn.functional.softmax(res, dim=1)[0] * 100
class_names[index[0]], percentage[index[0]].item()



('stop', 72.13072204589844)